<a href="https://colab.research.google.com/github/flohmannjr/tensorflow_curso/blob/main/TensorFlow_MP2_1_SkimLit%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Milestone Project 2.1: SkimLit+

Esta é a continuidade do projeto SkimLit, segundo milestone project do curso TensorFlow Developer Certificate in 2023: Zero to Mastery. https://www.udemy.com/share/104ssS3@m--2vC9jNX6kIapUf0IUOOxnuj8_1SisbHiZTsJDQ5z2toWaLfaenc4bvZ4i2Yd5/

# Importações

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.data import AUTOTUNE, Dataset
from tensorflow.keras import Model
from tensorflow.keras.layers import Bidirectional, Concatenate, Dense, Dropout, Embedding, Input, LSTM, TextVectorization
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Configurações, constantes e variáveis globais

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi'] = 100

plt.style.use('seaborn-darkgrid')

COR = '#007f66'

DIRETORIO = '/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign'

LIMITE_DICIONARIO = 68000
LIMITE_CARACTERES = 80

LIMITE_INCORPORADOR = 300
LIMITE_INCORPORADOR_CARACTERES = 25
LIMITE_VETOR_TEXTO = 100

LOTE_TAMANHO = 32

ENTRADA_FORMATO = (1,)
ENTRADA_TIPO = tf.string

ATIVACAO_CNN = 'relu'
ATIVACAO_RNN = 'tanh'
ATIVACAO_SAIDA = 'softmax'

SUAVIZACAO = 0.2
APRENDIZADO = 0.001
METRICAS = ['accuracy']

ITERACOES = 3

avaliacoes = [None] * 6

# Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/tensorflow_curso/main/funcoes.py

In [ ]:
from funcoes import avaliar_modelo, grafico_historico_por_iteracao, preprocessar_texto

# Dados

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git

In [ ]:
df_treino    = pd.DataFrame(preprocessar_texto(f'{DIRETORIO}/train.txt'))
df_validacao = pd.DataFrame(preprocessar_texto(f'{DIRETORIO}/dev.txt'))

codificador_onehot = OneHotEncoder(sparse=False)

rotulos_onehot_treino    = codificador_onehot.fit_transform(df_treino['classe'].to_numpy().reshape(-1, 1))
rotulos_onehot_validacao = codificador_onehot.transform(df_validacao['classe'].to_numpy().reshape(-1, 1))

codificador_int = LabelEncoder()

rotulos_int_treino    = codificador_int.fit_transform(df_treino['classe'].to_numpy())
rotulos_int_validacao = codificador_int.transform(df_validacao['classe'].to_numpy())

classes = codificador_int.classes_

palavras_por_texto_98   = int(np.percentile([len(texto.split()) for texto in df_treino['texto']], 98))
caracteres_por_texto_98 = int(np.percentile([len(texto) for texto in df_treino['texto']], 98))

lista_caracteres_treino    = [" ".join(list(texto)) for texto in df_treino['texto']]
lista_caracteres_validacao = [" ".join(list(texto)) for texto in df_validacao['texto']]

numero_linha_98 = int(np.percentile(df_treino['numero'], 98))

linhas_numero_onehot_treino    = tf.one_hot(df_treino['numero'], depth=numero_linha_98)
linhas_numero_onehot_validacao = tf.one_hot(df_validacao['numero'], depth=numero_linha_98)

total_linhas_98 = int(np.percentile(df_treino['total'], 98))

linhas_total_onehot_treino    = tf.one_hot(df_treino['total'], depth=total_linhas_98)
linhas_total_onehot_validacao = tf.one_hot(df_validacao['total'], depth=total_linhas_98)

dados_hibridos_linhas_treino_textos  = Dataset.from_tensor_slices((df_treino['texto'], lista_caracteres_treino, linhas_numero_onehot_treino, linhas_total_onehot_treino))
dados_hibridos_linhas_treino_rotulos = Dataset.from_tensor_slices(rotulos_onehot_treino)
dados_hibridos_linhas_treino         = Dataset.zip((dados_hibridos_linhas_treino_textos, dados_hibridos_linhas_treino_rotulos))
dados_hibridos_linhas_treino         = dados_hibridos_linhas_treino.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

dados_hibridos_linhas_validacao_textos  = Dataset.from_tensor_slices((df_validacao['texto'], lista_caracteres_validacao, linhas_numero_onehot_validacao, linhas_total_onehot_validacao))
dados_hibridos_linhas_validacao_rotulos = Dataset.from_tensor_slices(rotulos_onehot_validacao)
dados_hibridos_linhas_validacao         = Dataset.zip((dados_hibridos_linhas_validacao_textos, dados_hibridos_linhas_validacao_rotulos))
dados_hibridos_linhas_validacao         = dados_hibridos_linhas_validacao.batch(LOTE_TAMANHO).prefetch(AUTOTUNE)

# Camadas auxiliares

In [ ]:
vetorizador_palavras = TextVectorization(max_tokens=LIMITE_DICIONARIO,
                                         output_mode='int',
                                         output_sequence_length=palavras_por_texto_98,
                                         name='vetorizador_palavras')

vetorizador_palavras.adapt(df_treino['texto'])

vocabulario_palavras = vetorizador_palavras.get_vocabulary()

incorporador_palavras = Embedding(input_dim=len(vocabulario_palavras),
                                  output_dim=LIMITE_INCORPORADOR,
                                  mask_zero=True,
                                  input_length=palavras_por_texto_98,
                                  name='incorporador_palavras')

vetorizador_caracteres = TextVectorization(max_tokens=LIMITE_CARACTERES,
                                           output_mode='int',
                                           output_sequence_length=caracteres_por_texto_98,
                                           name='vetorizador_caracteres')

vetorizador_caracteres.adapt(lista_caracteres_treino)

vocabulario_caracteres = vetorizador_caracteres.get_vocabulary()

incorporador_caracteres = Embedding(input_dim=len(vocabulario_caracteres),
                                    output_dim=LIMITE_INCORPORADOR_CARACTERES,
                                    mask_zero=True,
                                    input_length=caracteres_por_texto_98,
                                    name='incorporador_caracteres')

incorporador_use = hub.KerasLayer(handle='https://tfhub.dev/google/universal-sentence-encoder/4',
                                  trainable=False,
                                  name='incorporador_use')

# Modelo 0: Modelo 5 do projeto original

## Modelo com incorporação de palavras

In [ ]:
modelo_nome = 'modelo_incorporacao_palavras'

entradas = Input(shape=[], dtype=ENTRADA_TIPO, name='camada_entrada_palavras')

incorporador_hub = incorporador_use(entradas)

saidas = Dense(LIMITE_INCORPORADOR, activation=ATIVACAO_CNN, name='camada_relu_palavras')(incorporador_hub)

modelo_incorporacao_palavras = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

## Modelo com incorporação de caracteres

In [ ]:
modelo_nome = 'modelo_incorporacao_caracteres'

entradas = Input(shape=ENTRADA_FORMATO, dtype=ENTRADA_TIPO, name='camada_entrada_caracteres')

camadas = vetorizador_caracteres(entradas)
camadas = incorporador_caracteres(camadas)

saidas = Bidirectional(layer=LSTM(units=LIMITE_INCORPORADOR_CARACTERES, activation=ATIVACAO_RNN), name='camada_bi_lstm_caracteres')(camadas)

modelo_incorporacao_caracteres = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

## Combinação dos modelos de incorporação

In [ ]:
combinacao_incorporacao = Concatenate(name="camada_combinacao_incorporacao")([modelo_incorporacao_palavras.output,
                                                                              modelo_incorporacao_caracteres.output])

## Camadas de abandono

In [ ]:
abandonos_incorporacao = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_incorporacao_relu')(combinacao_incorporacao)
abandonos_incorporacao = Dropout(rate=0.5, name='camada_abandonos_incorporacao')(abandonos_incorporacao)

## Modelo para o número de linhas

In [ ]:
modelo_nome = 'modelo_numero_linhas'

entradas = Input(shape=(numero_linha_98,), dtype=linhas_numero_onehot_treino.dtype, name='camada_entrada_numero_linhas')

saidas = Dense(numero_linha_98 * 2, activation=ATIVACAO_CNN, name='camada_relu_numero_linhas')(entradas)

modelo_numero_linhas = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

## Modelo para o total de linhas

In [ ]:
modelo_nome = 'modelo_total_linhas'

entradas = Input(shape=(total_linhas_98,), dtype=linhas_total_onehot_treino.dtype, name='camada_entrada_total_linhas')

saidas = Dense(total_linhas_98 * 2, activation=ATIVACAO_CNN, name='camada_relu_total_linhas')(entradas)

modelo_total_linhas = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

## Combinação dos modelos de incorporação e linhas

In [ ]:
combinacao_linhas = Concatenate(name="camada_combinacao_linhas")([abandonos_incorporacao,
                                                                  modelo_numero_linhas.output,
                                                                  modelo_total_linhas.output])

## Camadas de abandono e saída

In [ ]:
abandonos_linhas = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_linhas_relu')(combinacao_linhas)
abandonos_linhas = Dropout(rate=0.5, name='camada_abandonos_linhas')(abandonos_linhas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(abandonos_linhas)

## Modelo com incorporação híbrida e linhas

In [ ]:
modelo_nome = 'modelo_0_modelo_5_original'

modelo = Model(inputs=[modelo_incorporacao_palavras.input,
                       modelo_incorporacao_caracteres.input,
                       modelo_numero_linhas.input,
                       modelo_total_linhas.input],
               outputs=saidas,
               name=modelo_nome)

## Verificação do modelo

In [ ]:
modelo.summary()

In [ ]:
plot_model(modelo, show_shapes=True)

## Compilação com suavização de rótulo

In [ ]:
modelo.compile(loss=CategoricalCrossentropy(label_smoothing=SUAVIZACAO),
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

## Finalização do modelo

In [ ]:
historico = modelo.fit(dados_hibridos_linhas_treino,
                       epochs=ITERACOES,
                       validation_data=dados_hibridos_linhas_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_hibridos_linhas_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[0]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[0]

# Modelo 1: Substituição do incorporador de palavras

## Modelo com incorporação de palavras particularizada

In [ ]:
modelo_nome = 'modelo_incorporacao_palavras_particularizada'

entradas = Input(shape=[], dtype=ENTRADA_TIPO, name='camada_entrada_palavras')

camadas = vetorizador_palavras(entradas)
camadas = incorporador_palavras(camadas)

saidas = Bidirectional(layer=LSTM(units=int(LIMITE_INCORPORADOR / 2), activation=ATIVACAO_RNN), name='camada_bi_lstm_palavras')(camadas)

modelo_incorporacao_palavras_particularizada = Model(inputs=entradas, outputs=saidas, name=modelo_nome)

## Reuso de camadas anteriores ajustadas

In [ ]:
combinacao_incorporacao = Concatenate(name="camada_combinacao_incorporacao")([modelo_incorporacao_palavras_particularizada.output,
                                                                              modelo_incorporacao_caracteres.output])

abandonos_incorporacao = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_incorporacao_relu')(combinacao_incorporacao)
abandonos_incorporacao = Dropout(rate=0.5, name='camada_abandonos_incorporacao')(abandonos_incorporacao)

combinacao_linhas = Concatenate(name="camada_combinacao_linhas")([abandonos_incorporacao,
                                                                  modelo_numero_linhas.output,
                                                                  modelo_total_linhas.output])

abandonos_linhas = Dense(LIMITE_VETOR_TEXTO * 2, activation=ATIVACAO_CNN, name='camada_abandonos_linhas_relu')(combinacao_linhas)
abandonos_linhas = Dropout(rate=0.5, name='camada_abandonos_linhas')(abandonos_linhas)

saidas = Dense(len(classes), activation=ATIVACAO_SAIDA, name='camada_saida')(abandonos_linhas)

modelo_nome = 'modelo_1_incorporacao_palavras_particularizada'

modelo = Model(inputs=[modelo_incorporacao_palavras_particularizada.input,
                       modelo_incorporacao_caracteres.input,
                       modelo_numero_linhas.input,
                       modelo_total_linhas.input],
               outputs=saidas,
               name=modelo_nome)

modelo.compile(loss=CategoricalCrossentropy(label_smoothing=SUAVIZACAO),
               optimizer=Adam(learning_rate=APRENDIZADO),
               metrics=METRICAS)

## Verificação do modelo

In [ ]:
modelo.summary()

In [ ]:
plot_model(modelo, show_shapes=True)

## Finalização do modelo

In [ ]:
historico = modelo.fit(dados_hibridos_linhas_treino,
                       epochs=ITERACOES,
                       validation_data=dados_hibridos_linhas_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_hibridos_linhas_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[1]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[1]

# Modelo 2: Otimizador de descida de gradiente estocástica

## Reuso de camadas anteriores ajustadas

In [ ]:
modelo_nome = 'modelo_2_otimizador_sgd'

modelo = Model(inputs=[modelo_incorporacao_palavras_particularizada.input,
                       modelo_incorporacao_caracteres.input,
                       modelo_numero_linhas.input,
                       modelo_total_linhas.input],
               outputs=saidas,
               name=modelo_nome)

## Compilação com otimizador de descida de gradiente estocástica

In [ ]:
modelo.compile(loss=CategoricalCrossentropy(label_smoothing=SUAVIZACAO),
               optimizer=SGD(learning_rate=APRENDIZADO),
               metrics=METRICAS)

## Verificação do modelo

In [ ]:
modelo.summary()

In [ ]:
plot_model(modelo, show_shapes=True)

## Finalização do modelo

In [ ]:
historico = modelo.fit(dados_hibridos_linhas_treino,
                       epochs=ITERACOES,
                       validation_data=dados_hibridos_linhas_validacao,
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_hibridos_linhas_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[2]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[2]

# Modelo 3: Ajuste na taxa de aprendizado do otimizador

## Reuso de camadas anteriores ajustadas

In [ ]:
modelo_nome = 'modelo_3_ajuste_aprendizado'

modelo = Model(inputs=[modelo_incorporacao_palavras_particularizada.input,
                       modelo_incorporacao_caracteres.input,
                       modelo_numero_linhas.input,
                       modelo_total_linhas.input],
               outputs=saidas,
               name=modelo_nome)

## Ajuste na taxa de aprendizado do otimizador

In [ ]:
modelo.compile(loss=CategoricalCrossentropy(label_smoothing=SUAVIZACAO),
               optimizer=SGD(),
               metrics=METRICAS)

lrs = tf.keras.callbacks.LearningRateScheduler(lambda epoch: tf.add(0.001, tf.multiply(0.002, epoch)))

## Finalização do modelo

In [ ]:
historico = modelo.fit(dados_hibridos_linhas_treino,
                       epochs=6,
                       validation_data=dados_hibridos_linhas_validacao,
                       callbacks=[lrs],
                       verbose=1)

In [ ]:
grafico_historico_por_iteracao(historico)

In [ ]:
probabilidades = modelo.predict(dados_hibridos_linhas_validacao)
previsoes      = tf.argmax(probabilidades, axis=1)
avaliacoes[3]  = avaliar_modelo(rotulos_int_validacao, previsoes, classes)

In [ ]:
avaliacoes[3]

# Comparação de modelos

In [ ]:
avaliacoes = [a for a in avaliacoes if a is not None]

comparacao = pd.DataFrame(avaliacoes)
comparacao = comparacao.reset_index().melt(id_vars='index').rename(columns=str.title)

In [ ]:
sns.barplot(data=comparacao, x='Index', y='Value', hue='Variable', palette='summer_r')

plt.ylim(0.80, 0.90)

plt.title('Medidas de avaliação dos modelos')
plt.xlabel('Modelo')
plt.ylabel('Valor')

plt.legend(loc=(1.03, 0.76));

In [ ]:
comparacao_f1 = pd.DataFrame([a['pontuacao-f1'] for a in avaliacoes]).T

sns.barplot(data=comparacao_f1, color=COR)

plt.ylim(0.80, 0.90)

plt.title('Pontuação-F1')
plt.xlabel('Modelo')
plt.ylabel('');